# Missing Values Analysis & Imputation
In this simple exercise, we check a data set reporting data for chronic kidney disease available at 

https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

Check the original files for the information about the source. We are going to impute the data, save a version of the file without imputation in which all the examples with missing values are eliminated, a version of the imputed file, and another version with a flag attribute for each attribute that show whether an attribute was imputed or not.

## Relevant Information

The data contains 400 examples described by 25 attributes (24 actual attributes and one class). 11 attributes are nominal and 14 are numerical. 250 examples belongs to class ckd, 150 examples belong to class notckd.

## Attributes
- age		-	age	
- bp		-	blood pressure
- sg		-	specific gravity
- al		-   	albumin
- su		-	sugar
- rbc		-	red blood cells
- pc		-	pus cell
- pcc		-	pus cell clumps
- ba		-	bacteria
- bgr		-	blood glucose random
- bu		-	blood urea
- sc		-	serum creatinine
- sod		-	sodium
- pot		-	potassium
- hemo		-	hemoglobin
- pcv		-	packed cell volume
- wc		-	white blood cell count
- rc		-	red blood cell count
- htn		-	hypertension
- dm		-	diabetes mellitus
- cad		-	coronary artery disease
- appet		-	appetite
- pe		-	pedal edema
- ane		-	anemia
- class		-	class	

There are missing attribute values denoted by "?"

## Analysis
Let's start by loading the libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load the data in csv format
df = pd.read_csv("ChronicKidneyDiseaseFull.csv")

In [3]:
# just a quick look at the data
df.head()

age    bp     sg   al   su     rbc        pc         pcc          ba  \
0  48.0  80.0  1.020  1.0  0.0     NaN    normal  notpresent  notpresent   
1   7.0  50.0  1.020  4.0  0.0     NaN    normal  notpresent  notpresent   
2  62.0  80.0  1.010  2.0  3.0  normal    normal  notpresent  notpresent   
3  48.0  70.0  1.005  4.0  0.0  normal  abnormal     present  notpresent   
4  51.0  80.0  1.010  2.0  0.0  normal    normal  notpresent  notpresent   

     bgr  ...    pcv    wbcc  rbcc  htn   dm  cad  appet   pe  ane class  
0  121.0  ...   44.0  7800.0   5.2  yes  yes   no   good   no   no   ckd  
1    NaN  ...   38.0  6000.0   NaN   no   no   no   good   no   no   ckd  
2  423.0  ...   31.0  7500.0   NaN   no  yes   no   poor   no  yes   ckd  
3  117.0  ...   32.0  6700.0   3.9  yes   no   no   poor  yes  yes   ckd  
4  106.0  ...   35.0  7300.0   4.6   no   no   no   good   no   no   ckd  

[5 rows x 25 columns]

In [4]:
# stats for numerical attributes
df.describe()

age          bp          sg          al          su         bgr  \
count  391.000000  388.000000  353.000000  354.000000  351.000000  356.000000   
mean    51.483376   76.469072    1.017408    1.016949    0.450142  148.036517   
std     17.169714   13.683637    0.005717    1.352679    1.099191   79.281714   
min      2.000000   50.000000    1.005000    0.000000    0.000000   22.000000   
25%     42.000000   70.000000    1.010000    0.000000    0.000000   99.000000   
50%     55.000000   80.000000    1.020000    0.000000    0.000000  121.000000   
75%     64.500000   80.000000    1.020000    2.000000    0.000000  163.000000   
max     90.000000  180.000000    1.025000    5.000000    5.000000  490.000000   

               bu          sc         sod         pot        hemo         pcv  \
count  381.000000  383.000000  313.000000  312.000000  348.000000  329.000000   
mean    57.425722    3.072454  137.528754    4.627244   12.526437   38.884498   
std     50.503006    5.741126   10.408752    3.193904    2.912587    8.990105   
min      1.500000    0.400000    4.500000    2.500000    3.100000    9.000000   
25%     27.000000    0.900000  135.000000    3.800000   10.300000   32.000000   
50%     42.000000    1.300000  138.000000    4.400000   12.650000   40.000000   
75%     66.000000    2.800000  142.000000    4.900000   15.000000   45.000000   
max    391.000000   76.000000  163.000000   47.000000   17.800000   54.000000   

               wbcc        rbcc  
count    294.000000  269.000000  
mean    8406.122449    4.707435  
std     2944.474190    1.025323  
min     2200.000000    2.100000  
25%     6500.000000    3.900000  
50%     8000.000000    4.800000  
75%     9800.000000    5.400000  
max    26400.000000    8.000000

In [5]:
# stats for all the other attributes
df.describe(exclude=[np.number])

rbc      pc         pcc          ba  htn   dm  cad appet   pe  ane  \
count      248     335         396         396  398  398  398   399  399  399   
unique       2       2           2           2    2    2    2     2    2    2   
top     normal  normal  notpresent  notpresent   no   no   no  good   no   no   
freq       201     259         354         374  251  261  364   317  323  339   

       class  
count    400  
unique     2  
top      ckd  
freq     250

In [6]:
categorical_variables_mask = df.dtypes==object
categorical_variables = df.columns[categorical_variables_mask]
numerical_variables = df.columns[~categorical_variables_mask]
print("%d Categorical Variables\n%d Numerical Variables\n"%(len(categorical_variables),len(numerical_variables)))

11 Categorical Variables
14 Numerical Variables



In [7]:
unknown_per_columns = df.isnull().sum()
unknown_per_columns

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

## Basic solution: Don't impute
The simplest solution is to drop the examples or the variables that contain missing values. Let's try it and check how much our data are reduced. 

What happens if we delete the rows (the data points or examples) that have at least one missing value?

In [8]:
df_delete_rows = df.dropna(axis=0)
print("After cleaning, %d examples remain"%(df_delete_rows.shape[0]))

After cleaning, 158 examples remain


What if we delete the columns (the variables or features) that have at least one missing value?

In [9]:
df_delete_columns = df.dropna(axis=1)
print("After cleaning, %d variable remain"%(df_delete_columns.shape[1]))

After cleaning, 1 variable remain


Dropping examples or columns with missing values drastically reduces the amount of available data. Thus we need to impute. Since sklearn 0.19 cannot deal with categorical variables, we use the default imputer for numerical variables and an additional categorical imputer for nominal/categorical variables. Note that sklearn 0.20 will introduce a specific module sklearn.impute

First we impute the  numerical variables done using the median

In [10]:
from sklearn.preprocessing import Imputer
numerical_imputer = Imputer(missing_values="NaN",strategy="median", copy=True)
numerical_imputer.fit(df[numerical_variables])
df_numerical_imputed_array = numerical_imputer.transform(df[numerical_variables])
df_numerical_imputed = pd.DataFrame(data=df_numerical_imputed_array,columns=numerical_variables)

Next we imputate the categorical variable with a contributed imputer that has the same interface but works on single variables/columns

In [11]:
from sklearn_pandas import CategoricalImputer
categorical_data = {}
categorical_imputer = CategoricalImputer(missing_values="NaN")
for v in categorical_variables:
    categorical_imputer.fit(df[v])
    categorical_data[v] = categorical_imputer.transform(df[v])

df_categorical_imputed = pd.DataFrame(categorical_data,columns=categorical_variables)

In [12]:
# merge the two data frames
df_imputed = pd.concat([df_numerical_imputed,df_categorical_imputed],axis=1)

In [13]:
unknown_per_columns = df_imputed.isnull().sum()
unknown_per_columns

age      0
bp       0
sg       0
al       0
su       0
bgr      0
bu       0
sc       0
sod      0
pot      0
hemo     0
pcv      0
wbcc     0
rbcc     0
rbc      0
pc       0
pcc      0
ba       0
htn      0
dm       0
cad      0
appet    0
pe       0
ane      0
class    0
dtype: int64

We now have data without any missing values. We might decide to improve the imputation procedure using linear regression instead of the median to impute numerical values

In [14]:
df_delete_rows.to_csv("ChronicKidneyDiseaseWithMissing.csv", index=False)
df_imputed.to_csv("ChronicKidneyDiseaseImputed.csv", index=False)

In [15]:
df_imputed_with_flags = df_imputed.copy()
for v in df.columns:
    if v!='class':
        imputed_v = v + '_imputed'
        df_imputed_with_flags[imputed_v] = (df[v].isnull())*1

df_imputed_with_flags.to_csv("ChronicKidneyDiseaseImputedWithFlags.csv",index=False)